##### Importações:

In [1]:
import logging
from selenium.webdriver.remote.remote_connection import LOGGER
from selenium.common.exceptions import *
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.ui import WebDriverWait
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from parsel import Selector
from time import sleep
import random
import pandas as pd

##### Funções

Função que inicia o driver

In [2]:
def start_driver():
    options = Options()
    LOGGER.setLevel(logging.WARNING)
    # add arguments to options variable:
    arguments = ['--lang=pt-BR', '--start-maximized', '--disable-notifications'] #'--headless'
    for argument in arguments:
        options.add_argument(argument)

    driver = webdriver.Chrome(options=options)

    wait = WebDriverWait(
        driver,
        10,
        poll_frequency=1,
        ignored_exceptions=[
            NoSuchElementException,
            ElementNotVisibleException,
            ElementNotSelectableException
        ]
    )
    return driver, wait

Função para digitar naturalmente

In [3]:
def natural_type(text, field):
    for letter in text:
        field.send_keys(letter)
        sleep(random.randint(1,5)/30)


Função para abrir a página

In [14]:
def get_page(site, driver, wait):
    return driver.get(site)

Função para pesquisar as vagas

In [15]:
def search(driver, wait, text):
    wait.until(
        expected_conditions.visibility_of_all_elements_located(
        (By.XPATH, "//input[@id='text-input-what']")
        )
    )
    wait.until(
        expected_conditions.element_to_be_clickable(
            (By.XPATH, "//input[@id='text-input-what']")
        )
    )
    
    job_field = driver.find_element(By.XPATH, "//input[@id='text-input-what']")
    job_field.click()
    natural_type(text[0], job_field)
    sleep(1.2)
    location_field = driver.find_element(By.XPATH, "//input[@id='text-input-where']")
    location_field.click()
    sleep(1.8)
    natural_type(text[1], location_field)
    location_field.send_keys(Keys.ENTER)
    wait.until(
        expected_conditions.visibility_of_all_elements_located(
            (By.XPATH, "//td[@class='resultContent css-1qwrrf0 eu4oa1w0']")
        )
    )


Função para scraping

In [6]:
def scrape(driver, wait, base_url):
    response = Selector(text=driver.page_source)
    for vaga in response.xpath("//td[@class='resultContent css-1qwrrf0 eu4oa1w0']"):
        yield {
            "Cargo" : vaga.xpath("./div/h2/a/span/text()").get(),
            "Empresa" : vaga.xpath("./div[@class='company_location css-17fky0v e37uo190']//span[@data-testid='company-name']/text()").get(),
            "Local" : vaga.xpath("./div[@class='company_location css-17fky0v e37uo190']/div/div[@data-testid='text-location']/text()").get(),
            "Link" : base_url + vaga.xpath("./div/h2/a/@href").get()
        }

##### Função para scraping

In [7]:
def next_page(driver, wait):
    next_btn = driver.find_element(By.XPATH, "//a[@aria-label='Next Page']")
    driver.execute_script("arguments[0].scrollIntoView();", next_btn) 
    next_btn.click()
    wait.until(
        expected_conditions.element_to_be_clickable(
            (By.XPATH, "//td[@class='resultContent css-1qwrrf0 eu4oa1w0']")
        )
    )

##### Função main

In [18]:
def main():
    site = "https://br.indeed.com"
    text = ["Python", "Belo Horizonte, MG"]
    data = []
    driver, wait = start_driver()
    get_page(site, driver, wait)
    search(driver, wait, text)
    # iniciando o scraping...
    while True:
        page_data = scrape(driver, wait, site)
        data.extend(page_data)
        # pagination:
        try:
            next_page(driver, wait)
        except:
            break
    # salvando os dados:
    df = pd.DataFrame(data)
    df.to_json("vagas.json", orient='records', index=False)  


##### Executando...

In [19]:
main()

StaleElementReferenceException: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=125.0.6422.142); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7ED0E1F52+60322]
	(No symbol) [0x00007FF7ED05CEC9]
	(No symbol) [0x00007FF7ECF17EBA]
	(No symbol) [0x00007FF7ECF1DCEE]
	(No symbol) [0x00007FF7ECF20641]
	(No symbol) [0x00007FF7ECF206E0]
	(No symbol) [0x00007FF7ECF68392]
	(No symbol) [0x00007FF7ECF68472]
	(No symbol) [0x00007FF7ECF5E449]
	(No symbol) [0x00007FF7ECF8C21A]
	(No symbol) [0x00007FF7ECF5ADB6]
	(No symbol) [0x00007FF7ECF8C430]
	(No symbol) [0x00007FF7ECFABC80]
	(No symbol) [0x00007FF7ECF8BFC3]
	(No symbol) [0x00007FF7ECF59617]
	(No symbol) [0x00007FF7ECF5A211]
	GetHandleVerifier [0x00007FF7ED3F94AD+3301629]
	GetHandleVerifier [0x00007FF7ED4436D3+3605283]
	GetHandleVerifier [0x00007FF7E

Executando por partes: